In [283]:
'''
Atividade 3

(x) Executar uma simulação de horizonte infinito para estimar o tempo médio de espera em uma fila M/M/1.

(X) Eliminar o transiente usando a heurística MSER -5Y.

Estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.

A regra de parada é a precisão relativa de 5%.

Cenário I: λ =7 clientes por segundo; μ =10 clientes por segundo.

Cenário II: λ =8 clientes por segundo; μ =10 clientes por segundo.

Cenário III: λ =9 clientes por segundo; μ =10 clientes por segundo.

Cenário IV: λ =9,5 clientes por segundo; μ =10 clientes por segundo.

Plotar um gráfico comparativo dos quatro cenários.

Plotar uma linha horizontal para o valor teórico esperado em cada cenário.
#https://repository.lib.ncsu.edu/bitstream/handle/1840.16/6923/etd.pdf?sequence=2&isAllowed=y
'''

import numpy as np
import random
import timeit
import math
from scipy import stats

In [284]:
def G5(n):
    random.seed(0)

    vet_rn = []
    for i in range(0,n,1):
        vet_rn.append(random.random())
    
    return(vet_rn)

In [285]:
def random_list(vet,B):
    Ts = np.zeros(len(vet))
    for i in range(0,len(vet)):
        Ts[i] = -B*np.log(1-vet[i])
    return list(Ts)

In [286]:
def sum_intervals(vet):
    for i in range(1, len(vet)):
        vet[i] += vet[i-1]
    return vet

In [287]:
def calculate_confidence_95(mm1):
    return (stats.t.interval(confidence=0.95, df = len(mm1)-1, loc=np.mean(mm1), scale=stats.sem(mm1)))

In [288]:
def MSER5Y(tx_service):
    Z_values = np.zeros(k)

    for i in range(0, k):
        bff = 0
        for j in range(0,m):
            bff += tx_service[((m*i)+j)]

        Z_values[i] = bff/m

    return Z_values

def mean_MSER5Y(Z_values, d):
    bff = 0

    for i in range(d+1,len(Z_values)):
        bff += Z_values[i]
    bff = bff/(k-d)

    return bff

def var_MSER5Y(Z_values, d, mean):
    bff = 0

    for i in range(d+1,len(Z_values)):
        bff += (Z_values[i]-mean)**2
    bff = bff/(k-d)

    return bff

def MSER5(Z_values, k, d):    
    mean = mean_MSER5Y(Z_values, d)    
    var = var_MSER5Y(Z_values, d, mean)
    
    #print("d: ", d, "k: ", k,"\nMean Z: ", mean,"\nVar Z: ", var, "\n")
    
    bff = var/((k-d)**(1/2))

    return bff

In [289]:
def MSER5_cicle(tx_wait):
    d = 0
    Z_values = MSER5Y(tx_wait)

    MSER5_series = []

    while(d < (k/2)):

        MSER5_series.append(MSER5(Z_values, k, d))

        d += 1

    index_min = MSER5_series.index(np.amin(MSER5_series))
    return index_min*5

In [290]:
def process_queue(clock, wait, tx_service, tx_entry, qtd_iteracitons, wait_list, is_run):
    for i in range(0,len(tx_service)):
        bff = clock - tx_entry[i]
        wait += bff
        
        wait_list.append(wait)
    
        clock += tx_service[i]
        qtd_iteracitons += 1
        
    transiente = MSER5_cicle(wait_list)
    
    tx_service = tx_service[transiente:]
    tx_entry = tx_entry[transiente:]
    
    confidence = calculate_confidence_95(tx_service)
    
    Table_value=1074.679
    
    if(Table_value>=confidence[0] and Table_value<=confidence[1]): is_run = False
    
    return clock, wait, tx_service, tx_entry, qtd_iteracitons, wait_list, is_run

In [291]:
def start_cicle(X, H, U):
    #U(μ) --> taxa de entrada
    #H(λ) --> taxa de serviço
    
    U = 1/U
    H = 1/H
    
    clock = 0
    wait = 0
    qtd_iteracitons = 0
    wait_list = []
    is_run = True
    
    tx_service = []
    tx_entry = []
    
    N = 0
    
    inicio = timeit.default_timer()
    
    while(is_run):
        
        N += X
        m = 5
        k = int(N/m)
        
        random_arr = G5(X)
        tx_service = tx_service + random_list(random_arr,U)
       
        bff = random_list(random_arr,H)
        if(qtd_iteracitons == 0):
            bff[0] = 0
            tx_entry.append(0)
        else:
            bff = [tx_entry[len(tx_entry)-1]] + bff
        
        bff = sum_intervals(bff)
        tx_entry = tx_entry + bff[1:]
        
        clock, wait, tx_service, tx_entry, qtd_iteracitons, wait_list, is_run = process_queue(clock, wait, tx_service, tx_entry, qtd_iteracitons, wait_list, is_run)
        
    fim = timeit.default_timer()

    return (wait_time/qtd_iteracitons),(fim-inicio)

In [292]:
start_cicle(1000000, 7, 10)

KeyboardInterrupt: 